# Skin segmentation - Solution


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

In [ ]:
# load file from google drive
from google.colab import drive
drive.mount('/content/drive')
data = np.loadtxt('/content/drive/My Drive/DataScience2021/Skin_NonSkin.txt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load file from local dir
data = np.loadtxt('Skin_NonSkin.txt')

In [ ]:
# split params from output
rgb  = data[:,:3].astype('float32')
# 'normalize' values to {0 - noskin , 1 - skin}
lbl = 2-(data[:,3].astype('float32') )

rgb_t = torch.from_numpy(rgb)
lbl_t = torch.from_numpy(lbl).reshape(-1,1)

In [ ]:
# model = nn.Sequential(nn.Linear(3,100), nn.Sigmoid() ,nn.Linear(100,1) )
# model = nn.Sequential(nn.Linear(3,100), nn.ReLU(), nn.Linear(100,20), nn.SELU() ,nn.Linear(20,1) )

model = nn.Sequential(nn.Linear(3,64), nn.BatchNorm1d(64), nn.ReLU(), nn.Linear(64,64), nn.BatchNorm1d(64),nn.ReLU() ,nn.Linear(64,1) )

#model2.pt
#model = nn.Sequential(nn.Linear(3,64), nn.ReLU(), nn.Linear(64,64), nn.ReLU() ,nn.Linear(64,1) )

In [ ]:
dataset = torch.utils.data.TensorDataset(rgb_t, lbl_t)
train_dataset, validation_dataset = torch.utils.data.random_split(dataset,(200000,45057))
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=len(train_dataset))

optimizer = torch.optim.Adam(model.parameters(),lr = 0.003)

# loss_fn = torch.nn.MSELoss()
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
def accuracy(valid, pred): # percent of True Positive
  rounded = torch.round(torch.sigmoid(pred))
  correct = (rounded == valid).sum().float()
  #return torch.round((correct/(valid.shape[0]))*100)
  return (correct/(valid.shape[0])*100)

In [ ]:
%%time
print('%-5s %-12s %-12s %-12s' % ('epoch','loss','tAccuracy','vAccuracy'))
# E = 20
E = 30
for epoch in range(1,E+1): 
  for datum in train_loader:
    optimizer.zero_grad()
    (features,target) = datum
    pred = model(features)
    loss = loss_fn(pred, target)
    train_acc = accuracy(target, pred)

    valid_pred = model(validation_dataset[:][0])
    valid_acc = accuracy(validation_dataset[:][1], valid_pred)

    loss.backward()
    optimizer.step()
  if epoch%5==0:
    print(f"{epoch:<5d} {loss:<12.6f} {train_acc:<12.6f} {valid_acc:<12.6f}")
print(f"{epoch:<5d} {loss:<12.6f} {train_acc:<12.6f} {valid_acc:<12.6f}")

epoch loss         tAccuracy    vAccuracy   
5     0.380804     93.173500    93.261871   
10    0.263385     95.782501    95.838608   
15    0.203638     97.339996    97.385529   
20    0.163952     97.868500    97.904877   
25    0.134173     98.067497    98.137917   
30    0.110555     98.512497    98.606209   
30    0.110555     98.512497    98.606209   
CPU times: user 1min 27s, sys: 2.44 s, total: 1min 29s
Wall time: 1min 30s


In [ ]:
from sklearn.metrics import confusion_matrix

t_x = validation_dataset[:][0]
t_valid = validation_dataset[:][1]

t_pred = torch.round(torch.sigmoid(model(t_x)))

# Calculate Confusion Matrix
# From sklearn docs (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html):
# Thus in binary classification, the count of true negatives is C0,0, false negatives is C10 , true positives is C11 and false positives is C01.
#  Matrix
# | True Negative  , False Positive |
# | False Negative , True Positive  |

CM = confusion_matrix(t_valid.data.view(-1).numpy(),t_pred.data.view(-1).numpy())
# unpack to variables
(TN,FP),(FN,TP) = CM

# From wikipedia (https://en.wikipedia.org/wiki/Sensitivity_and_specificity)
#  true positive (TP) - A test result that correctly indicates the presence of a condition or characteristic
# false positive (FP) - A test result which wrongly indicates that a particular condition or attribute is present

TPR = TP/(TP+FN)

FPR = FP/(FP+TN)


print(f"""Model's
True Positive Rate: {TPR}
False Positive Rate: {FPR}""")


Model's
True Positive Rate: 0.9995761364840521
False Positive Rate: 0.017321729365524986


Model Saving

In [ ]:
model_id = 0

In [ ]:
print(model_id)

3


In [ ]:
# save to google drive
model_id += 1
model_file = f"/content/drive/My Drive/DataScience2021/models_skin/model{model_id}.pt"
print(f'Saving model at {model_file}')
torch.save(model, model_file)

Saving model at /content/drive/My Drive/DataScience2021/models_skin/model4.pt
